In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ravaligangineni","key":"0ad4fc7f895aae9450561c763fd108ff"}'}

In [10]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow import keras
from tensorflow.keras import layers


In [2]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c dogs-vs-cats

!unzip -qq dogs-vs-cats.zip
!unzip -qq test1.zip
!unzip -qq train.zip

100% 810M/812M [00:21<00:00, 42.5MB/s]
100% 812M/812M [00:21<00:00, 39.7MB/s]


In [6]:
import os, shutil, pathlib

original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")

def make_subset(subset_name, start_index, end_index):
    for category in ("cat", "dog"):
        dir = new_base_dir / subset_name / category
        os.makedirs(dir, exist_ok=True)  # Set exist_ok to True to avoid FileExistsError
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
        for fname in fnames:
            shutil.copyfile(src=original_dir / fname, dst=dir / fname)


In [ ]:
from tensorflow.keras.utils import image_dataset_from_directory

# Sample size 1000
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2000)

# Load datasets with descriptive names
training_data = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_data = image_dataset_from_directory(new_base_dir / "validation", image_size=(180, 180), batch_size=32)
testing_data = image_dataset_from_directory(new_base_dir / "test", image_size=(180, 180), batch_size=32)

# Train from scratch with data augmentation
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2)
])

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
history = model.fit(training_data, epochs=3, validation_data=validation_data)

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate(testing_data)
print(f"Test accuracy: {test_accuracy:.3f}")


In [12]:
test_loss, test_accuracy = model.evaluate(testing_data)
print(f"Test accuracy: {test_accuracy:.3f}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 18s 537ms/step - accuracy: 0.5669 - loss: 0.6838
Test accuracy: 0.590


In [14]:
# Increase training sample size to 1500
make_subset("train", start_index=0, end_index=1500)
make_subset("validation", start_index=1500, end_index=2000)
make_subset("test", start_index=2000, end_index=2500)

# Load datasets with larger training size
training_data_large = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_data_large = image_dataset_from_directory(new_base_dir / "validation", image_size=(180, 180), batch_size=32)

# Train with larger training set
history_large = model.fit(training_data_large, epochs=3, validation_data=validation_data_large)

# Evaluate on the test set
test_loss_large, test_accuracy_large = model.evaluate(testing_data)
print(f"Test accuracy with larger sample: {test_accuracy_large:.3f}")


Found 3000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.
Epoch 1/3
94/94 ━━━━━━━━━━━━━━━━━━━━ 241s 3s/step - accuracy: 0.6496 - loss: 0.6203 - val_accuracy: 0.6370 - val_loss: 0.6368
Epoch 2/3
94/94 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.6623 - loss: 0.6032 - val_accuracy: 0.6740 - val_loss: 0.5907
Epoch 3/3
94/94 ━━━━━━━━━━━━━━━━━━━━ 274s 3s/step - accuracy: 0.6740 - loss: 0.5986 - val_accuracy: 0.6425 - val_loss: 0.6440
32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 581ms/step - accuracy: 0.6195 - loss: 0.6946
Test accuracy with larger sample: 0.626


In [15]:
# Increase the training sample size to 2000 for optimal performance
make_subset("train", start_index=0, end_index=2000)
make_subset("validation", start_index=2000, end_index=2500)
make_subset("test", start_index=2500, end_index=3000)

# Load datasets
training_data_optimal = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_data_optimal = image_dataset_from_directory(new_base_dir / "validation", image_size=(180, 180), batch_size=32)

# Model with data augmentation and regularization
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2)
])

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(32, kernel_size=3, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(64, kernel_size=3, activation="relu", kernel_regularizer=keras.regularizers.l2(0.001))(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.6)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_optimal = keras.Model(inputs, outputs)

# Compile the model
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model_optimal.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Train the model for 3 epochs
history_optimal = model_optimal.fit(training_data_optimal, epochs=3, validation_data=validation_data_optimal)


Found 4000 files belonging to 2 classes.
Found 3000 files belonging to 2 classes.
Epoch 1/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 318s 3s/step - accuracy: 0.5223 - loss: 0.7380 - val_accuracy: 0.5253 - val_loss: 0.7163
Epoch 2/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 335s 3s/step - accuracy: 0.5817 - loss: 0.7084 - val_accuracy: 0.6287 - val_loss: 0.6755
Epoch 3/3
125/125 ━━━━━━━━━━━━━━━━━━━━ 318s 3s/step - accuracy: 0.6409 - loss: 0.6820 - val_accuracy: 0.6407 - val_loss: 0.6589


NameError: name 'test_dataset' is not defined

In [16]:
# Evaluate on the test set
test_loss_optimal, test_accuracy_optimal = model_optimal.evaluate(testing_data)
print(f"Test accuracy with optimized sample size: {test_accuracy_optimal:.3f}")


32/32 ━━━━━━━━━━━━━━━━━━━━ 19s 584ms/step - accuracy: 0.6191 - loss: 0.6779
Test accuracy with optimized sample size: 0.630


In [20]:
# Define the VGG16 base before the function
conv_base = keras.applications.vgg16.VGG16(weights="imagenet", include_top=False, input_shape=(180, 180, 3))

# Freeze the convolutional base to avoid training it
conv_base.trainable = False

# Define the get_features_and_labels function
def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)  # Preprocess for VGG16
        features = conv_base.predict(preprocessed_images)  # Extract features using VGG16's base
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
# Sample size 1000 for pretrained model
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1500)
make_subset("test", start_index=1500, end_index=2000)

# Load datasets
training_data_pretrained = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_data_pretrained = image_dataset_from_directory(new_base_dir / "validation", image_size=(180, 180), batch_size=32)
testing_data_pretrained = image_dataset_from_directory(new_base_dir / "test", image_size=(180, 180), batch_size=32)

# Extract features from the pretrained VGG16 base
train_features_pretrained, train_labels_pretrained = get_features_and_labels(training_data_pretrained)
val_features_pretrained, val_labels_pretrained = get_features_and_labels(validation_data_pretrained)
test_features_pretrained, test_labels_pretrained = get_features_and_labels(testing_data_pretrained)

# Fully connected classification layer on top of VGG16 base
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model_vgg_pretrained = keras.Model(inputs, outputs)

model_vgg_pretrained.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

# Train the model with 3 epochs for time efficiency
history_vgg_pretrained = model_vgg_pretrained.fit(train_features_pretrained, train_labels_pretrained, epochs=3, validation_data=(val_features_pretrained, val_labels_pretrained))

# Evaluate on the test set
test_loss_vgg_pretrained, test_accuracy_vgg_pretrained = model_vgg_pretrained.evaluate(test_features_pretrained, test_labels_pretrained)
print(f"Test accuracy with pretrained model (1000 samples): {test_accuracy_vgg_pretrained:.3f}")


Found 4000 files belonging to 2 classes.
Found 3000 files belonging to 2 classes.
Found 3000 files belonging to 2 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step
1/1

In [ ]:
# Sample size 1500 for pretrained model
make_subset("train", start_index=0, end_index=1500)
make_subset("validation", start_index=1500, end_index=2000)
make_subset("test", start_index=2000, end_index=2500)

# Load datasets with larger training size
training_data_pretrained_larger = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_data_pretrained_larger = image_dataset_from_directory(new_base_dir / "validation", image_size=(180, 180), batch_size=32)

# Extract features using the pretrained VGG16 model
train_features_pretrained_larger, train_labels_pretrained_larger = get_features_and_labels(training_data_pretrained_larger)
val_features_pretrained_larger, val_labels_pretrained_larger = get_features_and_labels(validation_data_pretrained_larger)

# Train the model with 3 epochs for the larger training sample
history_vgg_pretrained_larger = model_vgg_pretrained.fit(train_features_pretrained_larger, train_labels_pretrained_larger, epochs=3, validation_data=(val_features_pretrained_larger, val_labels_pretrained_larger))

# Evaluate on the test set
test_loss_vgg_pretrained_larger, test_accuracy_vgg_pretrained_larger = model_vgg_pretrained.evaluate(test_features_pretrained, test_labels_pretrained)
print(f"Test accuracy with pretrained model (1500 samples): {test_accuracy_vgg_pretrained_larger:.3f}")


In [ ]:
# Sample size 2000 for pretrained model (Optimal size)
make_subset("train", start_index=0, end_index=2000)
make_subset("validation", start_index=2000, end_index=2500)
make_subset("test", start_index=2500, end_index=3000)

# Load datasets with optimal training size
training_data_pretrained_optimal = image_dataset_from_directory(new_base_dir / "train", image_size=(180, 180), batch_size=32)
validation_data_pretrained_optimal = image_dataset_from_directory(new_base_dir / "validation", image_size=(180, 180), batch_size=32)

# Extract features using the pretrained VGG16 model
train_features_pretrained_optimal, train_labels_pretrained_optimal = get_features_and_labels(training_data_pretrained_optimal)
val_features_pretrained_optimal, val_labels_pretrained_optimal = get_features_and_labels(validation_data_pretrained_optimal)

# Train the model with 3 epochs for optimal training size
history_vgg_pretrained_optimal = model_vgg_pretrained.fit(train_features_pretrained_optimal, train_labels_pretrained_optimal, epochs=3, validation_data=(val_features_pretrained_optimal, val_labels_pretrained_optimal))

# Evaluate on the test set
test_loss_vgg_pretrained_optimal, test_accuracy_vgg_pretrained_optimal = model_vgg_pretrained.evaluate(test_features_pretrained, test_labels_pretrained)
print(f"Test accuracy with pretrained model (2000 samples): {test_accuracy_vgg_pretrained_optimal:.3f}")
